In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import warnings

logreg_anneal_control

In [2]:
def logreg_anneal_control(start=0, end=0, iter=0, earlyout=0, update=0):
    if isinstance(start, dict):
        if 'end' in start and len(start['end']) > 0:
            end = start['end']
        if 'iter' in start and len(start['iter']) > 0:
            iter = start['iter']
        if 'earlyout' in start and len(start['earlyout']) > 0:
            earlyout = start['earlyout']
        if 'update' in start and len(start['update']) > 0:
            update = start['update']
        if 'start' in start and len(start['start']) > 0:
            start = start['start']
        else:
            start = 0
    if iter < 100 and (start != 0 or end != 0):
        raise ValueError("not enough repetitions")
    if start < end:
        raise ValueError("starting temperature below ending temperature")
    return {'start': start, 'end': end, 'iter': iter, 'earlyout': earlyout, 'update': update}

logreg_tree_control

In [3]:
def logreg_tree_control(treesize=8, opers=1, minmass=0, n1=None):
    if isinstance(treesize, dict):
        if 'opers' in treesize and len(treesize['opers']) > 0:
            opers = treesize['opers']
        if 'minmass' in treesize and len(treesize['minmass']) > 0:
            minmass = treesize['minmass']
        if 'treesize' in treesize and len(treesize['treesize']) > 0:
            treesize = treesize['treesize']
        else:
            treesize = 16
    
    treesize = 2 ** np.floor(np.log2(np.abs(treesize + 0.0001)))
    
    if opers != 2 and opers != 3 and opers != 4:
        opers = 1
    
    if minmass < 0:
        minmass = 0
    
    if n1 is not None:
        if minmass > n1 / 4:
            raise ValueError("minmass should be at most samplesize/4")
    
    return {'treesize': treesize, 'opers': opers, 'minmass': minmass}

logreg_mc_control

In [4]:
def logreg_mc_control(nburn=1000, niter=25000, hyperpars=0, update=0, output=4):
    if isinstance(nburn, dict):
        if 'niter' in nburn and len(nburn['niter']) > 0:
            niter = nburn['niter']
        if 'hyperpars' in nburn and len(nburn['hyperpars']) > 0:
            hyperpars = nburn['hyperpars']
        if 'output' in nburn and len(nburn['output']) > 0:
            output = nburn['output']
        if 'update' in nburn and len(nburn['update']) > 0:
            update = nburn['update']
        if 'nburn' in nburn and len(nburn['nburn']) > 0:
            nburn = nburn['nburn']
        else:
            nburn = 1000
    hyperpars = np.concatenate([hyperpars, np.zeros(4)])[:4]
    return {'nburn': nburn, 'niter': niter, 'hyperpars': hyperpars, 'update': update, 'output': output}

# savefit data

In [5]:
root = 'C:/Users/asus/Desktop/CSC4050/LogicReg/R/savefit/'
tree1 = pd.read_csv(root+'savefit1_tree1.csv')
tree2 = pd.read_csv(root+'savefit1_tree2.csv')

response = pd.read_csv(root+'savefit1_response.csv')
response = response.values.squeeze()

censor = pd.read_csv(root+'savefit1_censor.csv')
censor = censor.values.squeeze()

weight = pd.read_csv(root+'savefit1_weight.csv')
weight = weight.values.squeeze()

binary = pd.read_csv(root+'savefit1_binary.csv', header=None, skiprows=[0])
binary = binary.values

logreg_savefit1 = {
    'class': 'logreg',
    'nsample': 500,
    'nbinary': 20,
    'nseparate': 0,
    'type': 'regression',
    'select': 'single.model',
    'anneal.control':{
        'start': -1,
        'end': -4,
        'iter': 25000,
        'earlyout': 0,
        'update': 1000,
    },
    'tree.control': {
        'treesize': 8,
        'opers': 1,
        'minmass': 0,
        'operators': 'both',
    },
    'seed': 0,
    'choice': 1,
    'nleaves': -1,
    'ntrees': 2,
    'penalty': 0,
    'response': response, # double[500] 
    'binary':  binary, # double[500*20]: 0 or 1
    'separate': 0, 
    'censor': censor, # double[500]
    'weight': weight, # double[500]
    'model': {
        'class': 'logregmodel',
        'ntrees': np.array([2, 2]), # dtype=np.float64
        'nleaves': np.array([-1, -1]), # dtype=np.float64
        'score': np.array([0.966493], dtype=np.float64),
        'coef': np.array([1.982887, -1.304364, 2.153445], dtype=np.float64),
        'trees': [
            {
                'class': 'logregtree',
                'whichtree': 1,
                'coef': np.array([-1.304364], dtype=np.float64),
                'trees': tree1.values, # A data.frame with 15 rows and columns(number,conc,knot,neg,pick)
            },
            {
                'class': 'logregtree',
                'whichtree': 2,
                'coef': np.array([2.153445], dtype=np.float64),
                'trees': tree2.values, # A data.frame with 15 rows and columns(number,conc,knot,neg,pick)
            },
        ]
    },
    # 'call': {
    #     'func': logreg,
    #     'args': {
    #         'resp': logreg_testdat[:,0],
    #         'bin': logreg_testdat[:,1:21],
    #         'type': 2,
    #         'select': 1,
    #         'ntrees': 2,
    #         'anneal_control': myanneal,
    #     }
    # }
}

# # apply 'call'
# logreg_savefit1['call']['func'](**logreg_savefit1['call']['args'])

eval_logreg

In [6]:
def eval_logreg(ltree, data):
   '''
   ltree: 'logreggtree' or 'logregmodel', part of result of an object of class logreg, generated with select=1 (single model fit), select=2 (multiple model fit),
   or select=6 (greedy stepwise fit).
   
   data: a data frame on which the logic tree is to be evaluated. Binary, have the same number of column as the bin component of the original logreg fit.
   '''
   if ltree['class'] == "logregtree": # using dictionary
      ltree = ltree['trees']  
      nn = np.ones((data.shape[0], ltree.shape[0])) 
      for i in range(ltree.shape[0]):
         if ltree[i, 1] == 3: # Check if node type is 3 (indicating a variable)
            nn[:, i] = data[:, ltree[i, 2]-1] # -1
            if ltree[i, 3] == 1: # 0: regular variable; 1: its complement
               nn[:, i] = 1 - nn[:, i] 
      ox = ltree.shape[0] # Backward pass through the trees for and and or nodes
      ox = ox // 2
      for i in range(ox-1, -1, -1):
         if ltree[i, 1] == 1: # Check if node type is 1 (indicating an and node) 
            nn[:, i] = nn[:, 2*(i+1)-1] * nn[:, 2*(i+1)]
         if ltree[i, 1] == 2: # Check if node type is 2 (indicating a or node)
            nn[:, i] = nn[:, 2*(i+1)-1] + nn[:, 2*(i+1)]
      nn = np.clip(nn, None, 1) # Set values greater than 1 to 1
      if np.count_nonzero(ltree) == 0: # Check if all nodes in ltree are zeros
         nn[:, 0] = 0 ##
      return nn[:, 0]
   
   else:
      if ltree['class'] == "logregmodel": # If ltree is an object of class logregmodel, recursively evaluate each tree
         n1 = eval_logreg(ltree['trees'][0], data)
         if ltree['ntrees'][0] > 1:
            for i in range(1, ltree['ntrees'][0]):
               n1 = np.column_stack((n1, eval_logreg(ltree['trees'][i], data))) # combine the results
         return n1
      else:
         raise ValueError("ltree not of class logregtree of logregmodel")

In [9]:
eval_logreg(ltree=logreg_savefit1['model'], data=logreg_savefit1['binary'])

array([[1., 1.],
       [0., 1.],
       [0., 0.],
       [0., 1.],
       [0., 0.],
       [0., 1.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 0.],
       [0., 1.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 0.],
       [0., 1.],
       [0., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 1.],
       [0., 0.],
       [0., 1.],
       [0., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 0.],
       [0., 1.

frame_logreg

In [7]:
def frame_logreg(fit, msz=None, ntr=None, newbin=None, newresp=None, newsep=None, newcens=None, newweight=None):
    
    if fit['class'] != "logreg": 
        raise ValueError("fit not of class logreg")

    outframe = pd.DataFrame() 
    # If 'newbin' is missing, use the original data
    if newbin is None: # missing(newbin)
        outframe['y'] = fit['response']  
        outframe['wgt'] = fit['weight'] 
        if fit['type'] == "proportional.hazards" or fit['type'] == "exponential.survival":
            outframe['cens'] = fit['censor']
        if fit['nseparate'] > 0: # nsep=?nseparate??? 
            outframe = pd.concat([outframe, fit['separate']], axis=1)
        binhere = fit['binary']

    else:
        binhere = newbin # if 'newbin' is specified
        # lbinhere = len(binhere)
        if isinstance(binhere, pd.DataFrame):
            binhere = binhere.values
        if not isinstance(binhere, np.ndarray):   # use array to substitude matrix
            binhere = np.array(binhere).reshape((-1,fit['nbinary']))
        
        n1 = binhere.shape[0]
        n2 = binhere.shape[1]

        if n2 != fit['nbinary']:
            raise ValueError("new number of binary predictors doesn't match fit")

        if newweight is not None:
            if len(newweight) != n1:
                raise ValueError("length(newweight) != length(newbin[,1])")
        else:
            newweight = np.repeat(1, n1)

        if newresp is not None: 
            if len(newresp) != n1:
                raise ValueError("length(newresp) != length(newbin[,1])")
            outframe['y'] = newresp
            outframe['wgt'] = newweight
        else:
            outframe['wgt'] = newweight

        if fit['type'] == "proportional.hazards" or fit['type'] == "exponential.survival": # newcens for proportional hazards models and exponential survival models only.
            if newcens is None:
                warnings.warn("Warning: newcens missing, taking all censoring indicators to be 1")
                outframe['cens'] = np.repeat(1, n1)
            else:
                if len(newcens) != n1:
                    raise ValueError("length(newcens) != length(newbin[,1])")
                outframe['cens'] = newcens

        if fit['nseparate'] > 0:
            if newsep is None:
                raise ValueError("you need to specify newsep")
            if not isinstance(newsep, np.ndarray): # use array to substitude matrix
                newsep = np.array(newsep).reshape((-1,fit['nseparate']))
            if isinstance(newsep, pd.DataFrame):
                newsep = newsep.values
            if newsep.shape[0] != n1:
                raise ValueError("length(newsep[,1]) != length(newbin[,1])")
            if newsep.shape[1] != fit['nseparate']:
                raise ValueError("new number of separate predictors doesn't match fit")
            outframe = pd.concat([outframe, pd.DataFrame(newsep)], axis=1)

    if fit['choice'] != 1 and fit['choice'] != 2 and fit['choice'] != 6: # check the type of model
        raise ValueError("fit$choice needs to be 1, 2, or 6")

    if fit['choice'] == 1:
        ntr = fit['ntrees']
        msz = fit['nleaves'] 
        for j in range(ntr):
            mtree = fit['model']['trees'][j]['trees']
            if msz < 0:
                msz = 0.5 * (mtree.shape[0]+1)
            if mtree[0, 1] == 0: # the first node is empty
                outframe['tmp'] = 0
            else:
                tmp = eval_logreg(fit['model']['trees'][j], binhere)
                outframe['tmp'] = tmp
            col_name = f"tree{msz}.{ntr}.{j}"
            outframe.rename(columns={"tmp": col_name}, inplace=True)

    else:
        for i in range(len(fit['nmodels'])): ## nmodels/alltrees
            xfit = fit['alltrees'][i]
            ntrx = xfit['ntrees']
            mszx = xfit['nleaves']
            l1 = 1
            
            if ntr is not None and fit['choice'] == 2:
                l1 = sum(ntrx == ntr)
            if msz is not None:
                l1 *=  sum(mszx == msz)
            if l1 > 0:
                for j in range(ntrx):
                    if xfit['trees'][j]['trees'][0, 1] == 0:
                        outframe['tmp'] = 0
                    else:
                        tmp = eval_logreg(xfit['trees'][j], binhere)
                        outframe['tmp'] = tmp
                    col_name = f"tree{mszx}.{ntrx}.{j}"
                    outframe.rename(columns={"tmp": col_name}, inplace=True)

    return outframe

In [8]:
frame_logreg(fit=logreg_savefit1)

,y,wgt,tree8.0.2.0,tree8.0.2.1
0,2.171474,1,1.0,1.0
1,3.904972,1,0.0,1.0
2,2.205592,1,0.0,0.0
3,3.692276,1,0.0,1.0
4,1.939746,1,0.0,0.0
...,...,...,...,...
495,0.626938,1,0.0,0.0
496,3.299834,1,0.0,1.0
497,5.651117,1,0.0,1.0
498,4.058269,1,0.0,1.0


predict_logreg

In [ ]:
def unstrip(x):
    dd = x.shape
    y = x
    if len(dd) == 2:
        dd2 = dd[1]
        if dd2 == 1:
            y = x[:, 0]
        if dd2 == 2:
            y = np.column_stack((x[:, 0], x[:, 1])) # to test
        if dd2 > 2:
            y = np.column_stack((x[:, 0], x[:, 1], x[:, 2]))
        if dd2 > 3:
            for i in range(3, dd2):
                y = np.column_stack((y, x[:, i]))
        return y

    if len(dd) == 1 or len(dd) == 0:
        y = np.array(x).flatten()
        # names(y) = NULL 移除列名
    return y


def predict_logreg(object, msz=None, ntr=None, newbin=None, newsep=None, **kwargs):
    if object['class'] != "logreg":
        raise ValueError("object not of class logreg")
        
    if object['choice'] > 2 and object['choice'] != 6:
        raise ValueError("object$choice needs to be 1, 2, or 6")

    if newbin is not None:
        if msz is None and ntr is None and newsep is None:
            y = frame_logreg(fit=object, newbin=newbin)
        elif msz is None and ntr is None and newsep is not None:
            y = frame_logreg(fit=object, newbin=newbin, newsep=newsep)
        elif msz is None and ntr is not None and newsep is None:
            y = frame_logreg(fit=object, newbin=newbin, ntr=ntr)
        elif msz is None and ntr is not None and newsep is not None:
            y = frame_logreg(fit=object, newbin=newbin, newsep=newsep, ntr=ntr)
        elif msz is not None and ntr is None and newsep is None:
            y = frame_logreg(fit=object, newbin=newbin, msz=msz)
        elif msz is not None and ntr is None and newsep is not None:
            y = frame_logreg(fit=object, newbin=newbin, newsep=newsep, msz=msz)
        elif msz is not None and ntr is not None and newsep is None:
            y = frame_logreg(fit=object, newbin=newbin, ntr=ntr, msz=msz)
        elif msz is not None and ntr is not None and newsep is not None:
            y = frame_logreg(fit=object, newbin=newbin, newsep=newsep, ntr=ntr, msz=msz)
    else:
        if msz is None and ntr is None:
            y = frame_logreg(fit=object)[:, 1:] # exclude the first column
        elif msz is None and ntr is not None:
            y = frame_logreg(fit=object, ntr=ntr)[:, 1:]
        elif msz is not None and ntr is None:
            y = frame_logreg(fit=object, msz=msz)[:, 1:]
        elif msz is not None and ntr is not None:
            y = frame_logreg(fit=object, ntr=ntr, msz=msz)[:, 1:]

    iik = 0
    if msz is None and ntr is None and object['select'] == "greedy":
        iik = 1

    ly = len(y[:, 0]) #y=500
    if not isinstance(y, np.ndarray):
        y = np.reshape(unstrip(y), (ly, -1), order='F')

    lly = len(y[:, 0])
    y = y[:, 1:]
    if lly == 1:
        y = np.array(y).reshape(1, -1) 

    if object['type'] == "proportional.hazards":
        y = y[:, 1:]

    z = None
    if len(y) == ly:
        y = np.array(y).reshape(-1, 1) 
    
    y = np.column_stack((y, np.array([1]*ly))) 

    if object['select'] == "single.model":
        z = np.array([object['model']['coef'][0]] * len(y[:, 0])) # array([coef, coef, ..., coef])
        for i in range(1, len(object['model']['coef'])):
            z += object['model']['coef'][i] * y[:, i - 1]

    if object['select'] == "multiple.models" or object['select'] == "greedy":
        if msz is None:
            msz = range(min(object['nleaves']), max(object['nleaves']))
        if ntr is None:
            ntr = range(min(object['ntrees']), max(object['ntrees']))
        z = None

        if object['nseparation'] > 0:
            y1 = y[:, :object['nseparation']]
            y = y[:, object['nseparation']:]

        jk = 0
        for i in range(object['nmodels']):
            nt1 = object['alltrees'][i]['ntrees']
            ms1 = object['alltrees'][i]['nleaves']

            if sum(nt1 == ntr) * sum(ms1 == msz) > 0 or iik == 1:
                jk += 1
                if ly == 1:
                    y = np.array(y).reshape(1, -1) 
                else:
                    if len(y) == ly:
                        y = np.array(y).reshape(-1, 1) 
                
                y3 = y[:, :nt1]
                if i != object['nmodels']:
                    y = y[:, nt1:]

                if nt1 == 1:
                    y3 = np.array(y3).reshape(-1, 1) 

                if object['nseparation'] > 0:
                    y3 = np.column_stack((y1, y3)) ##

                cc = object['alltrees'][i]['coef']
                z2 = cc[0]
                if len(cc) > 1:
                    for ii in range(1, len(cc)):
                        z2 += cc[ii] * y3[:, ii - 1]

                str_v = f"tr{nt1}.lf{ms1}"
                if z is not None:
                    z = df = pd.DataFrame({'z': z, 'z2': z2})
                else:
                    z = pd.DataFrame({'z2': z2})

                z.columns[jk-1] = str_v 

        if object['type'] == "classification":
            z[z != 0] = 1

        if object['type'] == "logistic":
            z = np.exp(z) / (1 + np.exp(z))

    return z

logreg

In [ ]:
# in logreg function

def logreg_binary(x):
    l1 = x.shape[0]*x.shape[1]
    l2 = np.sum(x == 0)
    l3 = np.sum(x == 1)
    return l1 == (l2 + l3)

def logreg_storetree(x): #input: numpy array
    i1 = np.array(x[3:], dtype=float).reshape(-1, 4)
    i2 = i1.shape[0]
    i3 = pd.DataFrame({'number': np.arange(1, i2 + 1),
                       'conc': i1[:, 0],
                       'knot': i1[:, 1],
                       'neg': i1[:, 2],
                       'pick': i1[:, 3]})
    return i3